##Я - тетрадка, которая всё запускает

Dependencies:
* Python 2, Ipython notebook
* Docker, nshou/elasticsearch-kibana
* numpy, ROOT, root_numpy, zmq


How to launch a ES+kibana container:
* sudo docker run -d -p 9200:9200 -p 5601:5601 nshou/elasticsearch-kibana

Howto use kibana directly (non embedded):
* kibana висит на localhost:5601
* После первого запуска кибана попросит Configure an index pattern
 * убрать галочку index contains time-based events
 * поставить дефолтный паттерн * (или просто по вкусу), 
 * нажать "Create"
* Как построить простую гистограмму
 * наверху тык Visualize -> Vertical Bar Chart -> from new search
 * в buckets тык X Axis; 
 * Aggregation <- Histogram
 * Field <- интересующее поле (например, muonHits)
 * Interval <- 1 (или сколько хочется отдать по X на 1 бин гистограммы)
 * тык зелёную кнопку Play (треугольник) выше
* а вообще https://www.elastic.co/guide/en/kibana/current/getting-started.html
  


###запуск сервера zmq
* просто кладёт входные пакеты в очередь и отдаёт их по мере просьб "get"
* периодически меняет run number
* клиент-обёртка zmq_client.py
* zmq.REP

In [1]:
import os
os.system('python zmq_event_bus.py &')

0

###Запуск клиента-складывателя всего в ES
* процесс-индексатор, который читает события пакетами из zmq_event_bus и пишет в ElasticSearch
* Имя индекса - "run"+run_number
* Недостающие индексы создаются

In [2]:
import indexer

In [3]:
inder = indexer.Indexer(verbose=True)
#начинаем складывать эвенты в индексы. 
inder.start_indexing()

requesting run id...
run id recieved: 15000
processing events...
672 events indexed in current run_id
processing events...
1580 events indexed in current run_id
processing events...
3670 events indexed in current run_id
processing events...
3932 events indexed in current run_id
processing events...
4668 events indexed in current run_id
processing events...
5112 events indexed in current run_id
run id recieved: 15001
processing events...
1134 events indexed in current run_id
processing events...
2487 events indexed in current run_id
processing events...
4042 events indexed in current run_id
processing events...
4853 events indexed in current run_id
processing events...
5443 events indexed in current run_id
run id recieved: 15002
processing events...
1219 events indexed in current run_id
processing events...
2274 events indexed in current run_id
processing events...
3945 events indexed in current run_id
processing events...
4199 events indexed in current run_id
run id recieved: 15003
pro

KeyboardInterrupt: 

###Алсо
* каждое событие пишется в индекс внутри ElasticSearch
* id события - порядковый номер его получения внутри run_id (todo: мб eventNumber?)
* маппинг события:

In [4]:
import event
event.es_event_mapping

{'_source': {'enabled': False},
 'properties': {'avgMass': {'index': 'not_analyzed', 'type': 'float'},
  'hcalEnergy': {'index': 'not_analyzed', 'type': 'float'},
  'muonHits': {'index': 'not_analyzed', 'type': 'float'}}}

#sanity check

In [5]:
import elasticsearch
es = elasticsearch.Elasticsearch(["localhost:9200"])

In [6]:
es.search()

{u'_shards': {u'failed': 0, u'successful': 66, u'total': 66},
 u'hits': {u'hits': [{u'_id': u'4.1.2',
    u'_index': u'.kibana',
    u'_score': 1.0,
    u'_source': {u'buildNum': 7562, u'defaultIndex': u'*'},
    u'_type': u'config'},
   {u'_id': u'*',
    u'_index': u'.kibana',
    u'_score': 1.0,
    u'_source': {u'fields': u'[{"name":"_source","type":"_source","count":0,"scripted":false,"indexed":false,"analyzed":false,"doc_values":false},{"name":"_index","type":"string","count":0,"scripted":false,"indexed":false,"analyzed":false,"doc_values":false},{"name":"muonHits","type":"number","count":0,"scripted":false,"indexed":true,"analyzed":false,"doc_values":false},{"name":"avgMass","type":"number","count":0,"scripted":false,"indexed":true,"analyzed":false,"doc_values":false},{"name":"_type","type":"string","count":0,"scripted":false,"indexed":true,"analyzed":false,"doc_values":false},{"name":"_id","type":"string","count":0,"scripted":false,"indexed":false,"analyzed":false,"doc_values":